In [1]:
import math

In [4]:
ATTRIBUTES = [
    "AAGE",
    "ACLSWKR",
    "ADTIND",
    "ADTOCC",
    # "AGI",
    "AHGA",
    "AHRSPAY",
    "AHSCOL",
    "AMARITL",
    "AMJIND",
    "AMJOCC",
    "ARACE",
    "AREORGN",
    "ASEX",
    "AUNMEM",
    "AUNTYPE",
    "AWKSTAT",
    "CAPGAIN",
    "CAPLOSS",
    "DIVVAL",
    # "FEDTAX",
    "FILESTAT",
    "GRINREG",
    "GRINST",
    "HHDFMX",
    "HHDREL",
    # "MARSUPWT",
    "MIGMTR1",
    "MIGMTR3",
    "MIGMTR4",
    "MIGSAME",
    "MIGSUN",
    "NOEMP",
    "PARENT",
    # "PEARNVAL",
    "PEFNTVTY",
    "PEMNTVTY",
    "PENATVTY",
    "PRCITSHP",
    # "PTOTVAL",
    "SEOTR",
    # "TAXINC",
    "VETQVA",
    "VETYN",
    "WKSWORK",
    "YEAR",
    "RESULT"
]

In [15]:
with open('census-income.data', 'r') as f:
    DATA = f.read().splitlines()
    DATA = [x.split(', ') for x in DATA]
    # drop the weight column
    DATA = [x[:24]+x[25:] for x in DATA]
# From names file
NUM_ATTRIBUTES = 40

In [16]:
zipped = list(zip(DATA[0],ATTRIBUTES))
print('\t\t'.join(f"{l[1]}: {l[0]}" for l in zipped))


AAGE: 73		ACLSWKR: Not in universe		ADTIND: 0		ADTOCC: 0		AHGA: High school graduate		AHRSPAY: 0		AHSCOL: Not in universe		AMARITL: Widowed		AMJIND: Not in universe or children		AMJOCC: Not in universe		ARACE: White		AREORGN: All other		ASEX: Female		AUNMEM: Not in universe		AUNTYPE: Not in universe		AWKSTAT: Not in labor force		CAPGAIN: 0		CAPLOSS: 0		DIVVAL: 0		FILESTAT: Nonfiler		GRINREG: Not in universe		GRINST: Not in universe		HHDFMX: Other Rel 18+ ever marr not in subfamily		HHDREL: Other relative of householder		MIGMTR1: ?		MIGMTR3: ?		MIGMTR4: ?		MIGSAME: Not in universe under 1 year old		MIGSUN: ?		NOEMP: 0		PARENT: Not in universe		PEFNTVTY: United-States		PEMNTVTY: United-States		PENATVTY: United-States		PRCITSHP: Native- Born in the United States		SEOTR: 0		VETQVA: Not in universe		VETYN: 2		WKSWORK: 0		YEAR: 95		RESULT: - 50000.


In [10]:
def partition(dataset:list[list], atr:int) -> dict[list[list]]:
    all_lists = dict()
    for point in dataset:
        val = point[atr]
        if val not in all_lists:
            all_lists[val] = [point]
        else:
            all_lists[val].append(point)
    return all_lists

def H_num(pos:int, neg:int):
    if (pos == 0 or neg == 0):
        return 0
    return -1*(pos/(pos+neg))*math.log((pos/(pos+neg)),2) - (neg/(pos+neg)) * math.log((neg/(pos+neg)),2)

def result_partition(dataset:list[list]):
    pos_list = []
    neg_list = []
    for point in dataset:
        if point[40] == '- 50000.':
            neg_list.append(point)
        else:
            pos_list.append(point)
    return (pos_list, neg_list)

def H(inlist:list):
    pos_list, neg_list = result_partition(inlist)
    pos, neg = len(pos_list), len(neg_list)
    return H_num(pos,neg)

def H_many(lists: list[list]):
    total_len = sum(len(l) for l in lists)
    total = 0
    for l in lists:
        total += H(l) * (len(l) / total_len)
    return total


In [138]:
print(f"Whole dataset: E = {H(DATA):.4f}")
results = [(ATTRIBUTES[i],H_many(partition(DATA, i).values())) for i in range(NUM_ATTRIBUTES)]
results.sort(key=lambda x: x[1])
print(results[:5])

Whole dataset: E = 0.3356
[('HHDREL', 0.14624299588736347), ('ADTOCC', 0.24301600254861988), ('AHGA', 0.2546131441375053), ('AMJOCC', 0.2585425745589341), ('VETYN', 0.2779912663787357)]


## Find single best 

In [18]:
start_e = H(DATA)
best_pick = -1
best_gain = 0
for i in range(NUM_ATTRIBUTES):
    gain = start_e - H_many(partition(DATA, i).values())
    # print(f"{ATTRIBUTES[i]}: E = {gain:.4f}")
    if gain > best_gain:
        best_gain = gain
        best_pick = i
print(f"Best pick: {ATTRIBUTES[best_pick]} with gain {best_gain:.4f}")

Best pick: ADTOCC with gain 0.0925


In [40]:
class Model:
    def __init__(self):
        self.tree=dict()
    def predict(self, tree, point):
        '''Tree is dict or string'''
        if type(tree) != dict:
            return tree
        else:
            attr = list(tree.keys())[0]
            val = point[attr]
            return self.predict(tree[attr][val], point)
    def training_error(self):
        correct_count = 0
        wrong_count = 0
        for point in DATA:
            if self.predict(self.tree, point) == point[40]:
                correct_count += 1
            else:
                wrong_count += 1
        return (correct_count / (correct_count + wrong_count))



In [143]:
def get_next_best_atr(dataset:list[list]):
    '''Returns best pick, attributes'''
    start_e = H(dataset)
    best_pick = -1
    best_pick_attributes = []
    best_gain = 0
    for i in range(NUM_ATTRIBUTES):
        partitioned_data = partition(dataset, i)
        gain = start_e - H_many(partitioned_data.values())
        if gain > best_gain:
            best_gain = gain
            best_pick = i
            best_pick_attributes = partitioned_data.keys()
    return best_pick, best_pick_attributes

def get_data_partition(dataset:list[list], atr:int, value):
    return partition(dataset, atr)[value]

## Actual Training

In [144]:
def rec_train(dataset:list[list], tree:dict, max_depth:int, depth:int=1):
    if depth > max_depth:
        return 
        
    next_pick, atr_values = get_next_best_atr(dataset)
    if next_pick == -1:
        return

    tree[next_pick]=dict()
    for atr_val in atr_values:
        partitioned_data = get_data_partition(dataset, next_pick, atr_val)
        # If we have a perfect class, make a leaf node
        if (H(partitioned_data) == 0):
            tree[next_pick][atr_val] = partitioned_data[0][40]
        elif (depth+1) > max_depth:
            # We are not allowed to go deeper, so make a guess at what anything here is instead of making a dict
            pos_list, neg_list = result_partition(partitioned_data)
            tree[next_pick][atr_val] = '+ 50000.' if len(pos_list) > len(neg_list) else '- 50000.'
        else:
            tree[next_pick][atr_val] = dict()
            rec_train(partitioned_data, tree[next_pick][atr_val], max_depth, depth+1)
            # If we have a -1, make a leaf node
            if len(tree[next_pick][atr_val]) == 0:
                pos_list, neg_list = result_partition(partitioned_data)
                tree[next_pick][atr_val] = '+ 50000.' if len(pos_list) > len(neg_list) else '- 50000.'


In [145]:
MAX_DEPTH = 2
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)


In [154]:
MAX_DEPTH = 2
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 2, error was 0.06256421565433556


In [89]:
MAX_DEPTH = 3
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 3, error was 0.041113054635305235


In [104]:
MAX_DEPTH = 4
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 4, error was 0.02552587922194438


In [146]:
MAX_DEPTH = 5
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 5, error was 0.015933000205490044


In [148]:
# Conflict Example
partitioned_data = get_data_partition(DATA, 3, '35')
partitioned_data = get_data_partition(partitioned_data, 18, '0')
partitioned_data = get_data_partition(partitioned_data, 2, '25')
partitioned_data = get_data_partition(partitioned_data, 0, '32')

print(f"Whole dataset: E = {H(partitioned_data):.4f}")
results = [(ATTRIBUTES[i],H_many(partition(partitioned_data, i).values())) for i in range(NUM_ATTRIBUTES)]
results.sort(key=lambda x: x[1])

print(results[:5])
print(partitioned_data[0])
print(partitioned_data[1])

Whole dataset: E = 1.0000
[('AAGE', 1.0), ('ACLSWKR', 1.0), ('ADTIND', 1.0), ('ADTOCC', 1.0), ('AHGA', 1.0)]
['32', 'Private', '25', '35', 'Bachelors degree(BA AB BS)', '0', 'Not in universe', 'Married-civilian spouse present', 'Manufacturing-nondurable goods', 'Precision production craft & repair', 'White', 'All other', 'Male', 'Not in universe', 'Not in universe', 'Children or Armed Forces', '0', '0', '0', 'Joint both under 65', 'Not in universe', 'Not in universe', 'Householder', 'Householder', 'Nonmover', 'Nonmover', 'Nonmover', 'Yes', 'Not in universe', '6', 'Not in universe', 'United-States', 'United-States', 'United-States', 'Native- Born in the United States', '0', 'Not in universe', '2', '52', '94', '50000+.']
['32', 'Private', '25', '35', 'Bachelors degree(BA AB BS)', '0', 'Not in universe', 'Married-civilian spouse present', 'Manufacturing-nondurable goods', 'Precision production craft & repair', 'White', 'All other', 'Male', 'Not in universe', 'Not in universe', 'Children o

In [150]:
MAX_DEPTH = 6
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 6, error was 0.007718408404043697


In [152]:
MAX_DEPTH = 7
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 7, error was 0.0027064548949243816


In [153]:
MAX_DEPTH = 8
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 8, error was 0.0012730361913162458


In [155]:
MAX_DEPTH = 9
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 9, error was 0.0008019125614591172


In [156]:
MAX_DEPTH = 10
curr_data = DATA[:]

model = Model()
rec_train(curr_data, model.tree, MAX_DEPTH)

print(f"For depth {MAX_DEPTH}, error was {1 - model.training_error()}")

For depth 10, error was 0.0005563268395122334
